# Project 2
## Members
* Ethan Kamus
* Nathaniel Marquez
* Rebecca Lee

In [25]:
import numpy as np
data = np.loadtxt('dataset.csv',delimiter=',',skiprows=1)

In [2]:
training_set = data[0:900]
test_set = data[901:1000]

In [57]:
# # remove the y column
arr = np.array(training_set)
# A = np.delete(A, 2, axis=1) # x-axis = 0, y-axis = 1
x1 = arr[0:900,[0]] # matrix of features?
x2 = arr[0:900,[1]]
X = np.hstack((np.ones_like(x1),x1,x2))
t = arr[0:900,[2]] # matrix of output?


# # compute weights
# for i in range(len(A)):
#     temp = np.array([A[i],A[i+1]])
# #     print(temp)
#     print(np.linalg.inv(temp))

# XX = np.dot(X.T,X)
# invXX = np.linalg.inv(XX)
# Xt = np.dot(X.T,t)
# w = np.dot(invXX,Xt)

XX = np.dot(X.T,X)
Xt = np.dot(X.T,t)
w = np.linalg.solve(XX,Xt)
t

array([[1.63247668e+00],
       [1.03137162e+00],
       [2.32614926e-01],
       [5.95721449e-01],
       [9.54343115e-01],
       [1.93129280e-01],
       [5.86145188e-02],
       [6.50621987e-01],
       [8.98436273e-01],
       [1.43419076e+00],
       [1.02033857e+00],
       [5.87537846e-01],
       [1.12381558e-01],
       [4.03915542e-01],
       [6.33194429e-01],
       [5.76038344e-01],
       [4.62759844e-01],
       [6.36082334e-01],
       [8.58722611e-01],
       [3.54439297e-01],
       [1.02797170e+00],
       [9.99757117e-01],
       [1.41714124e+00],
       [8.74362421e-01],
       [1.53933921e+00],
       [3.05416450e-01],
       [8.67378249e-01],
       [3.27701744e-01],
       [1.18463316e+00],
       [4.08831614e-01],
       [1.98657928e-01],
       [3.57655609e-01],
       [7.50747401e-01],
       [1.04326800e+00],
       [3.42270116e-01],
       [1.19763734e+00],
       [4.19835910e-01],
       [5.60659383e-01],
       [9.05976042e-02],
       [2.41655269e-01],


$$ y = 0.65683428 - 0.00632862w_1 - 0.00483534w_2$$

In [89]:
w_0 = w[0][0]
w_1 = w[1][0]
w_2 = w[2][0]
X = arr[0:900,[0,1]]
predictions = []
for i in X:
    predictions.append(np.array([w_0 + (i[0]*w_1) + (i[1]*w_2)]))
# predictions = np.array(predictions)
loss = t - predictions
average = np.sum(loss)/len(loss)
average

-4.460629396716185e-16